In [1]:
import pandas
import requests
import json
import re

In [2]:
books = pandas.read_csv('../../Sources/AI/bookai/Books.csv', 
                        names = ['ISBN', 'title', 'author', 'year_publication', 'publisher', 'img_s', 'img_m', 'img_l'], 
                        skiprows = 1, 
                        low_memory=False)

In [6]:
urlMultipleInsert = "http://localhost:8888/api/book/multiple"

def parseTitle (title):
    b = {}
    
    match1 = re.search("^(.*) \(((.*) (Series)?)\)", title)
    if match1:
        b["book_name"] = match1[1]
        b["series_name"] = match1[2]
    
    match2 = re.search("^(.*) \((.*), Book (\d)+\)", title)
    if match2:
        b["book_name"] = match2[1]
        b["series_name"] = match2[2]
        b["series_position"] = match2[3]
        
    if not match1 and not match2:
        b["book_name"] = title

    return b

def bookArray (index):
    b_data = parseTitle(books.title[index])
    b_data["isbn13"] = books.ISBN[index]
    b_data["authors"] = [books.author[index]]
    b_data["cover"] = books.img_l[index]
    
    return b_data

def getXBooks (x, start_i):
    lst = []
    for i in range(x):
        try:
            lst.append(bookArray(start_i + i))
        except:
            print("Error at index", start_i + i)
    return lst



nb_books = len(books.index)
print("Number of books: ", nb_books)

Number of books:  271360


In [4]:
def sendRequest (url, data):
    response = requests.post(url, json=data)
    if response.status_code != 200:
        print(data)
        print(response.text)

In [7]:
step = 20
start = 0
for i in range(5):
    books_data = getXBooks(step, start)
    sendRequest(urlMultipleInsert, books_data)
    start = start + step

[{'book_name': 'Classical Mythology', 'isbn13': '0195153448', 'authors': ['Mark P. O. Morford'], 'cover': 'http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg'}, {'book_name': 'Clara Callan', 'isbn13': '0002005018', 'authors': ['Richard Bruce Wright'], 'cover': 'http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg'}, {'book_name': 'Decision in Normandy', 'isbn13': '0060973129', 'authors': ["Carlo D'Este"], 'cover': 'http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg'}, {'book_name': 'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It', 'isbn13': '0374157065', 'authors': ['Gina Bari Kolata'], 'cover': 'http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg'}, {'book_name': 'The Mummies of Urumchi', 'isbn13': '0393045218', 'authors': ['E. J. W. Barber'], 'cover': 'http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg'}, {'book_name': "The Kitchen God's Wife", 'isbn13': '0399135782', 'authors': ['Amy T